In [1]:
import struct
import numpy as np
from PIL import Image
from pathlib import Path
import os
import shutil
import traceback
import sys
import geopandas as gpd
from tqdm import tqdm
import subprocess
import re
from pyproj import transform, Proj
from shapely import geometry, affinity
from rasterio.control import GroundControlPoint
from rasterio.transform import from_gcps
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from osgeo import gdal, osr; gdal.UseExceptions()
import pickle
from IPython.display import clear_output
import warnings; warnings.filterwarnings("ignore")
import importlib
def recreate_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path

def load_config(path):
    spec = importlib.util.spec_from_file_location("CFG", path)
    CFG = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(CFG)
    return CFG

In [2]:
DATA_DIR = "data/20221220_Sudol"
CFG = load_config(f"{DATA_DIR}/config.py").CONVERSION

In [3]:
RJPG_DIR = f"{DATA_DIR}/rjpg"
TIF_UNCAL_DIR = f"{DATA_DIR}/tif_uncal"
TMP_DIR = f"{DATA_DIR}/tmp"

# R-JPG to TIF

In [4]:
distance = round(CFG.DISTANCE)
humidity = round(CFG.HUMIDITY)
emissivity = round(CFG.EMISSIVITY,2)
reflection = round(CFG.REFLECTION)

assert os.path.exists(RJPG_DIR)
recreate_dir(TIF_UNCAL_DIR)
recreate_dir(TMP_DIR)
# get list of .jpg files from rjpg directory
file_list = list(filter(lambda x:x.endswith((".jpg", ".JPG")), os.listdir(RJPG_DIR)))
# iterate over files
for i, file_name in enumerate(file_list):
    clear_output(wait=True)
    print(f"File {i+1}/{len(file_list)}")
    #get raw bytes from rjpg file
    os.system(  f"./dji_irp -a measure \
                --measurefmt float32 \
                --distance {distance} \
                --humidity {humidity} \
                --emissivity {emissivity} \
                --reflection {reflection} \
                -s {RJPG_DIR}/{file_name} -o {TMP_DIR}/bytes.raw")
    img = Image.open(f"{RJPG_DIR}/{file_name}")
    # decode temporary byte file to tif
    arr = np.zeros(img.size[0]*img.size[1])
    with open(f"{TMP_DIR}/bytes.raw", "rb") as f:
        data = f.read()
        format = '{:d}f'.format(len(data)//4)
        arr=np.array(struct.unpack(format, data))
    arr = arr.reshape(img.size[1],img.size[0])
    im = Image.fromarray(arr)
    tif_file_name = file_name.replace('.JPG','.tif')
    im.save(f"{TIF_UNCAL_DIR}/{tif_file_name}")
    # copy exif data from original file to new tif
    os.system(f"exiftool -tagsfromfile {RJPG_DIR}/{file_name} {TIF_UNCAL_DIR}/{tif_file_name} -overwrite_original_in_place")

File 469/469
DIRP API version number : 0x13
DIRP API magic version  : d4c7dea
R-JPEG file path : data/20221220_Sudol/rjpg/DJI_20221220113309_0075_T.JPG
R-JPEG version information
    R-JPEG version : 0x1
    header version : 0x103
 curve LUT version : 0x1
R-JPEG resolution size
      image  width : 640
      image height : 512
Change distance from 5 to 25
Change humidity from 70 to 93
Change emissivity from 1 to 0.95
Change reflection from 23 to 0
Measurement: get params range:
distance: [1,25]
humidity: [20,100]
emissivity: [0.1,1]
reflection: [-40,500]
Run action 1
Save image file as : data/20221220_Sudol/tmp/bytes.raw
Test done with return code 0
    1 image files updated


# Cleanup

In [5]:
#clear temporary files
shutil.rmtree(TMP_DIR)